In [1]:
import os
from typing import List, Optional
import fire
from llama import Llama, Dialog
import llama
from tabulate import tabulate

In [2]:
# Define major elements of Llama2 7b
os.environ['models_loc'] : str          =       'D:\GitHub repositories\llama\\'
os.environ['ckpt_dir']   : str          =       os.environ['models_loc'] + 'llama-2-7b-chat' 
os.environ['tokenizer_path'] : str      =       os.environ['models_loc'] + 'tokenizer.model'
os.environ['RANK']                      =       '0'
os.environ['WORLD_SIZE']                =       '1'
os.environ['MASTER_ADDR']               =       'localhost'
os.environ['MASTER_PORT']               =       '12355'
# Another type of initialization
# torch.distributed.init_process_group(
#     backend='nccl',
#     init_method='env://',
#     timeout=datetime.timedelta(0, 1800),
#     world_size=0,
#     rank=0,
#     store=None,
#     group_name=''
# )


In [3]:
# Main parameters of the model for further tuning
# for now, we leave some of them as default
temperature : float     =   0.6
top_p : float           =   0.9
max_seq_len : int       =   4096
max_batch_size : int    =   3 # 16
max_gen_len : Optional[int] =   None

In [4]:
# We are now Building a model, which takes a bit of time of course
generator = Llama.build(
    ckpt_dir=os.environ['ckpt_dir'],
    tokenizer_path=os.environ['tokenizer_path'],
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
)


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 69.46 seconds


In [5]:
# Its time to print first results
def printastable( dialog_ : Dialog , lastchatonly : bool = False ) -> None:
    sc_results = list()
    for index , dialog in enumerate(dialog_):
        if not lastchatonly:
            for msg in dialog:
                sc_results.append( ( index , msg['role'].capitalize() , msg['content'].capitalize() ) )
        else:
            for msg in dialog[-2::]:
                sc_results.append( ( index , msg['role'].capitalize() , msg['content'].capitalize()  ) )
    print( tabulate( sc_results , headers=["Scenario" , "Role" , "Content"],tablefmt='rounded_grid' , maxcolwidths=[None,None,150]) )

In [6]:
# This function will append 'assistant' to the sequence of dialog, meaning that we are focusing on one instead of many ( there are alternative solutions however, but for now its convenient).
def Update( dialogs_ : list[Dialog], results_ ) -> None:
    for dialog , result in zip(dialogs_ , results_ ):
        assert result['generation']['role'] == "assistant"
        dialog.append(
                {
                    "role"      :   result['generation']['role'],
                    "content"   :   result['generation']['content'].capitalize()
                }
        )
# This function simply asks a question and return latest dialog and its result
# Unfortunately, due to the structure of generation function, a question must always be asked of all dialogs
def Ask( dialogs_ : list[Dialog] , question_ : str ) -> None:
    for dialog in dialogs_:
        assert dialog[-1]['role'] != "user" , f"ASK: {dialog[-1]['role']}"
        dialog.append(
                {
                "role"      :   "user",
                "content"   :   question_
                }
        )
    Update( dialogs_ , generator.chat_completion( dialogs_ , max_gen_len=max_gen_len , temperature=temperature , top_p=top_p ) )

## Scenario 1

On the first scenario, we are going to simply ask the pre-trained model two different questions, both of which are related to lithium, which is crucial in energy transition. Our aim is to inspect the capability of the model ( of course we know the godness of it! but what about Critical Raw Materials and Minerals ) and answer the following questions:
- Does the model acknowledge basic information about lithium?
- What is the general idea of the model about lithium industry?
- How the responses change by adding real information or fantasies?

In [7]:
# We need to make dialogs for the bot
SC1_Q1 : list[Dialog] = [
 
    [ # No information
        {
            "role"      :   "system",
            "content"   :   ""
        }
    ],
    [
            # From Wikipedia
        {   "role"      :   "system",
            "content"   :   """\
                Lithium is a chemical element with the symbol 'Li' and atomic number 3. It is a soft, silvery-white alkali metal.
                Under standard conditions, it is the least dense metal and the least dense solid element.
                Like all alkali metals, lithium is highly reactive and flammable, and must be stored in vacuum, inert atmosphere, or inert liquid such as purified kerosene or mineral oil.
                It exhibits a metallic luster. It corrodes quickly in air to a dull silvery gray, then black tarnish.
                It does not occur freely in nature, but occurs mainly as pegmatitic minerals, which were once the main source of lithium.
                Due to its solubility as an ion, it is present in ocean water and is commonly obtained from brines.
                Lithium metal is isolated electrolytically from a mixture of lithium chloride and potassium chloride.
                """
        }
    ],
    [
            # From Wikipedia
            # Wang, D. H., Dai, H. Z., Liu, S. B., Wang, C. H., Yu, Y., Dai, J. J., ... & Ma, S. C. (2020). Research and exploration progress on lithium deposits in China. China Geology, 3(1), 137-152.
        {   "role"      :   "system",
            "content"   :   """\
                Lithium is a chemical element with the symbol 'Li' and atomic number 3. It is a soft, silvery-white alkali metal.
                Under standard conditions, it is the least dense metal and the least dense solid element.
                Like all alkali metals, lithium is highly reactive and flammable, and must be stored in vacuum, inert atmosphere, or inert liquid such as purified kerosene or mineral oil.
                It exhibits a metallic luster. It corrodes quickly in air to a dull silvery gray, then black tarnish.
                It does not occur freely in nature, but occurs mainly as pegmatitic minerals, which were once the main source of lithium.
                Due to its solubility as an ion, it is present in ocean water and is commonly obtained from brines. Lithium metal is isolated electrolytically from a mixture of lithium chloride and potassium chloride. Also, if you are asked about geological example in china, add this as an example:
                The rare mineral resources including Li, Be, Nb and Ta are abundant in western Sichuan Province. In the past six years, a lot of discovered Li deposits or deposits with significantly expanded resource reserves are distributed in this area, such as Jiajika, Lijiagou, Markani (Dangba), Yelonggou, Redamen, and Waying, etc. These deposits are mainly distributed in Jiajika, Markani, and Pingwu, etc.. The amount of prospective resources can reach 7 Mt.
                """
        }
    ],
]

In [8]:
Ask( SC1_Q1 , "At which geographical locations we could expect the presense of lithium for exploration and mining?" )

In [9]:
printastable( SC1_Q1 )

╭────────────┬───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   Scenario │ Role      │ Content                                                                                                                                                │
├────────────┼───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│          0 │ System    │                                                                                                                                                        │
├────────────┼───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│          0 │ User      │ At which geographical locations we could expect the presense of lithium f

In [10]:
# Lets try with different question
SC1_Q2 : list[Dialog] = [
        [
            # No prior information
            {
                "role"      :   "system",
                "content"   :   ""
            }
        ],
        [
            # Some personal rought imprecise information
            {
                "role"      :   "system",
                "content"   :   """The market of lithium will grow roughly 20 percent by the end of 2040, with some geopolitical disruptions around 2030 in which lithium will lose its value by 5 percent."""
            }
        ],
        [
            # Mixture of imprecise and verified information
            {
                "role"      :   "system",
                "content"   :   """The market of lithium will grow roughly 20 percent by the end of 2040, with some geopolitical disruptions around 2030 in which lithium will lose its value by 5 percent. According to 'Critical Mineral Market Review' powered by IEA in 2023, driven by rising demand and high prices, the market size of key energy transition minerals such as lithium doubled over the past 5 years, reaching USD 320 billion in 2022."""
            }
        ]
]
Q2  =   "Forecast the growth and stability of lithium hydroxide and carbonate market for the next 37 years, please ( output format should be as a table)."

In [11]:
# Repeating the same procedure of generating and printing
Ask( SC1_Q2 , Q2 )

In [12]:
printastable( SC1_Q2 )

╭────────────┬───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   Scenario │ Role      │ Content                                                                                                                                                │
├────────────┼───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│          0 │ System    │                                                                                                                                                        │
├────────────┼───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│          0 │ User      │ Forecast the growth and stability of lithium hydroxide and carbonate mark